# Desafio #9

### Instalação de libs requeridas

In [ ]:
!pip install opencv-python imutils

In [ ]:
# Libs de apoio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import os , random , json , requests
import types
import pandas as pd

# Libs para criação de modelo DeepLearning
# ATENÇÃO - Não utilizar o Keras interno do Tensorflow por incompatibilidade com o WML
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import InputLayer
from keras.preprocessing import image as Kimage
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from imutils import paths

# Libs para comunicação APIs Watson IBM
from ibm_botocore.client import Config
import ibm_boto3
from watson_machine_learning_client import WatsonMachineLearningAPIClient

### Download do Dataset(imagens) da competição

In [ ]:
!wget -O dataset102.zip https://www.dropbox.com/s/trcr1k2dfmk8b44/DatasetBHTC.zip?dl=0

!unzip -o ./dataset102.zip
!mkdir DATASET
!mv CLEAN DIRTY DATASET

### Definição do Modelo - Não alterar o input_shape

In [ ]:
random.seed(1)
model = Sequential()
model.add(Conv2D(16, (3,3), input_shape=(96,96,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(16, (3,3), padding='same', activation='relu'))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

### Load das imagens, já com pre-processing, para utilizar no Train do modelo

In [ ]:
imagePaths = sorted(list(paths.list_images('DATASET/')))

random.seed(1)
random.shuffle(imagePaths)

data = []
labels = []

for imagePath in imagePaths:
    image = Kimage.load_img(imagePath,target_size=(96,96))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

### Split do dataset em Train e Test

In [ ]:
classes = ['CLEAN', 'DIRTY']
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
data = np.array(data, dtype="float") / 255.0

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=1)

print("shape de X de treino :",trainX.shape)
print("shape de Y de treino :",trainY.shape)
print("shape de X de teste :",testX.shape)
print("shape de Y de teste :",testY.shape)

### Treinamento do modelo - utilizando DataGenerator

In [ ]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
random.seed(1)
model.fit_generator(
    aug.flow(trainX, trainY, batch_size=16),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // 16,
    epochs=40)

### Gráficos para avaliação do modelo

In [ ]:
H=model.history
plt.rcParams['figure.figsize'] = 16, 8
plt.figure()
N = H.epoch[-1]+1
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

### Save do modelo como model.h5

In [ ]:
model.save("model_WSTUDIO.h5")

### Upload do modelo no IBM Cloud Object Storage

### Upload do modelo no IBM Watson Machine Learning - para export de API

In [ ]:
# o modelo deve estar em formato .tar.gz para subir no WML
!tar -zcvf model_WSTUDIO.tar.gz model_WSTUDIO.h5

In [ ]:
# Credenciais do Watson Machine Learning
wml_credentials  = {
  "apikey": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
  "iam_apikey_description": "Auto-generated for key XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX::serviceid:ServiceId-XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX",
  "instance_id": "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX",
  "url": "https://us-south.ml.cloud.ibm.com"
}
client = WatsonMachineLearningAPIClient( wml_credentials )

# Definição de metadados do modelo (versao de python, framework, libs e etc)
sample_saved_model_filename = 'model_WSTUDIO.tar.gz'
metadata = {
    client.repository.ModelMetaNames.NAME              : 'MY_FIRST_SUBMIT',
    client.repository.ModelMetaNames.FRAMEWORK_NAME    : 'tensorflow',
    client.repository.ModelMetaNames.FRAMEWORK_VERSION : '1.11',
    client.repository.ModelMetaNames.RUNTIME_NAME      : 'python',
    client.repository.ModelMetaNames.RUNTIME_VERSION   : '3.6',
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES :  [{"name": "keras", "version": "2.2.4"}]
}


# Conexão com o WML
model_details = client.repository.store_model( sample_saved_model_filename, meta_props=metadata, training_data=None )

# Deploy do modelo
model_id = model_details["metadata"]["guid"]
model_deployment_details = client.deployments.create( artifact_uid=model_id, name="MY FIRST SUBMIT D9 Behind The Code" )

# Retrieve da URL da API para consumo da mesma
model_endpoint_url = client.deployments.get_scoring_url( model_deployment_details )
print("A URL de chamada da sua API é : ",model_endpoint_url)

### Chamada da API para teste

In [ ]:
# Download de imagem teste
!wget -O teste2.jpg https://www.dropbox.com/s/73fofwe566749sl/teste123.jpg?dl=0

In [ ]:
# Montagem da estrutura de JSON para chamada da API do WML
ai_parms = { "wml_credentials" : wml_credentials, "model_endpoint_url" : model_endpoint_url }

# Load da imagem de teste e pre-processing da mesma - para entrada na rede neural convolucional
image = Kimage.load_img("teste2.jpg")
plt.imshow(image)
image = image.resize(size=(96,96))
image = img_to_array(image)
image = np.array(image, dtype="float") / 255.0
image = np.expand_dims(image,axis=0)
image = image.tolist()

# Chamada da função SCORE no modelo (inference)
model_payload = { "values" : image }
model_result = client.deployments.score( ai_parms["model_endpoint_url"], model_payload )
print(model_result)

print("\nImagem Classificada como : ", classes[model_result['values'][0][1][0]])

print("\nProbabilidades : \n\t",
      classes[model_result['values'][0][1][0]]," : %.2f" %(model_result['values'][0][0][0]*100),"%\n\t",
     )

# BOA SORTE